# WASD

# Actors
    Actors are the elements that perform actions within the simulation, and they can affect other actors.

    They include vehicles, walkers, sensors, traffic signs, traffic lights and the spectator.

In [1]:
!pip install carla

In [2]:
import carla
import math
import random
import time

# client : requires two arguments
    first : ip address of the machine where you are running carla server, in this case, my local machine.
    second : the port on which carla server is running, 2000 is the default

In [21]:
client=carla.Client('localhost', 2000)

# get_world
        
    allows to access all within the simulation so accessible vehicles, buildings, pedestrians, and other things in the map such as trees, traffic lights
        

In [4]:
# it also allows us to add or spawn these objects

world=client.get_world()

# The Blueprint Library (carla.BlueprintLibrary) is a summary of all carla.ActorBlueprint 
    blueprints = [bp for bp in world.get_blueprint_library().filter('*')]
    
    These layouts allow the user to smoothly incorporate new actors into the simulation.
    They are already-made models with animations and a series of attributes.
    
    These attributes include vehicle color, amount of channels in a lidar sensor, a walker's speed, and much more

In [5]:
# the blue print includes vehicles, boxes, kiosk or things you might find in the street, pedestrians

bp_lib=world.get_blueprint_library()

# Spawning
    The world object is responsible of spawning actors and keeping track of these. 
    Spawning only requires a blueprint, and a carla.Transform stating a location and rotation for the actor.

### spawn_actor(self, blueprint, transform, attach_to=None, attachment=Rigid) : 
    shows some sensors being attached to a car when spawned

    spawn_actor() raises an exception if the spawning fails.
    try_spawn_actor() returns None if the spawning fails.

# get_map
    3D location which are set in sensible locations on the road so you can spawn your vehicles on the road that is pointing in the right direction.
    
    Returns a list of recommendations made by the creators of the map to be used as spawning points for the vehicles. 
    
    The list includes carla.Transform objects with certain location and orientation. 

In [6]:
spawn_points=world.get_map().get_spawn_points()

In [7]:
# find the vehicle blueprint from the blueprint library
# lets specifying a specific vehicle which I want to use 'lincoln mkz_2020'

vehicle_bp=bp_lib.find('vehicle.lincoln.mkz_2020')

In [8]:
# using the 'world object' to 'try_spawn_actor' method to spawn that specific vehicle
# because when you spawn vehicles within simulation, it's not always necessarily going to work
# Try spawning the vehicle at a randomly chosen spawn point

vehicle=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# carla.Transform(self, location, rotation) 
    Class that defines a transformation, a combination of location and rotation, without scaling.

### spectator is the camera 

In [9]:
# lets move the vehicle to the specific location which is going to take us just above and behind the car that I've just spawned

spectator=world.get_spectator()

# it will move the spectator to see right behind the vehicle that I just spawned so that I can see it 

transform=carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [10]:
# add some more vehicles, randomly select vehicles from the blueprint library
for i in range(30):
    vehicle_bp=random.choice(bp_lib.filter('vehicle')) 
    npc=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

### Autopilot mode will subscribe a vehicle to the Traffic Manager to simulate real urban conditions. 
### This module is hard-coded, not based on machine learning.

In [11]:
# by this, it will hand over control of each vehicles to the traffic manager and they will move around the city

for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [12]:
# recording data using sensors

camera_bp=bp_lib.find('sensor.camera.rgb')
camera_init_trans=carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) #record the data

### listen(self, callback) 
### The function the sensor will be calling to every time a new measurement is received.

In [31]:
# start the camera saving data to disk
cc = carla.ColorConverter.CityScapesPalette
camera.listen(lambda image:image.save_to_disk('output/%06d.png' % image.frame, cc))

In [32]:
# Stop the camera when we've recorded enough data
camera.stop()

In [39]:
# client.start_recorder("recording01.log")

'C:/Users/USER/workspace/carla/carla/Unreal/CarlaUE4/Saved/recording01.log'

In [34]:
# client.stop_recorder()

In [38]:
# client.show_recorder_file_info("recording01.log")

ArgumentError: Python argument types in
    Client.show_recorder_file_info(Client, str)
did not match C++ signature:
    show_recorder_file_info(class carla::client::Client {lvalue}, class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > name, bool show_all)

In [37]:
# client.replay_file("./carla/Unreal/CarlaUE4/Saved/recording01.log")

ArgumentError: Python argument types in
    Client.replay_file(Client, str)
did not match C++ signature:
    replay_file(class carla::client::Client {lvalue}, class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > name, double time_start, double duration, unsigned int follow_id, bool replay_sensors=False)